# Heterostructure

1. 寻找原胞
2. 读取原胞
3. 判断角度的错配度
4. 判断晶格的错配度
5. 输出异质结搭配方案（原子数、晶格参数、晶格夹角）
6. 识别是否是bulk并自动切面
7. 构建异质结
8. 异质结横向建模，输入平移矢量（-1/3, 1/3, 0）

In [55]:
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
from tkinter import _flatten
import numpy as np
import spglib

def find_primitive_cell(atoms):
    cell = np.array(atoms.get_cell())
    scaled_positions = np.array(atoms.get_scaled_positions())
    atomic_numbers = np.array(atoms.get_atomic_numbers())
    pbc = np.array(atoms.get_pbc())
    origin_cell = (cell, scaled_positions, atomic_numbers)
    primitive_cell = spglib.standardize_cell(origin_cell, to_primitive=1, no_idealize=1)
    primitive_atoms = Atoms(numbers = primitive_cell[2],
                            scaled_positions = primitive_cell[1],
                            cell = primitive_cell[0],
                            pbc=pbc)
    return primitive_atoms

def read_data(file):
    atoms = read(file)
    natoms = len(atoms)
    basis_vector = atoms.cell[:]
    
    a_length = np.linalg.norm(basis_vector[0, :])
    b_length = np.linalg.norm(basis_vector[1, :])
    angle = np.arccos(np.dot(basis_vector[0, :2], basis_vector[1, :2]) / (np.linalg.norm(basis_vector[0, :2]) * np.linalg.norm(basis_vector[1, :2])))
    
    vector1 = np.array([np.cos(angle / 2) * a_length, np.sin(angle / 2) * a_length, 0])
    vector2 = np.array([np.cos(angle / 2) * b_length, -np.sin(angle / 2) * b_length, 0])
    
    return natoms, vector1, vector2

def save_data(l1, l2, dic, norm_veca, norm_vecb, norm_vecc, norm_vecd, it, filename):
    fp = open(filename, mode='w')
    qw = [0, 0, 0, 0, 0]
    int_width = 5 
    float_width = 15 
    num_decimals = 6   
    for i in it:
        m = dic['mm'][0][i]
        n = dic['nn'][0][i]
        qw[0] = abs(l1[m, 6] - l2[n, 6])
        qw[1] = abs(l1[m, 4] - l2[n, 4]) / l1[m, 4]
        qw[2] = abs(l1[m, 4] - l2[n, 4]) / l2[n, 4]
        qw[3] = abs(l1[m, 5] - l2[n, 5]) / l1[m, 5]
        qw[4] = abs(l1[m, 5] - l2[n, 5]) / l2[n, 5]
        fp.write(str(int(l1[m, 0])).rjust(int_width, ' '))
        fp.write(str(int(l1[m, 1])).rjust(int_width, ' '))
        fp.write(str(int(l1[m, 2])).rjust(int_width, ' '))
        fp.write(str(int(l1[m, 3])).rjust(int_width, ' '))
        fp.write(str(int(l2[n, 0])).rjust(int_width, ' '))
        fp.write(str(int(l2[n, 1])).rjust(int_width, ' '))
        fp.write(str(int(l2[n, 2])).rjust(int_width, ' '))
        fp.write(str(int(l2[n, 3])).rjust(int_width, ' '))
        fp.write(str(qw[0]).rjust(float_width + 10, ' '))
        fp.write(str(np.around(l1[m, 6], 1)).rjust(float_width - 5, ' '))
        fp.write(str(np.round(l2[n, 6], 1)).rjust(float_width - 5, ' '))
        fp.write(str(np.around(qw[1], num_decimals)).rjust(float_width, ' '))
        fp.write(str(np.round(qw[2], num_decimals)).rjust(float_width, ' '))
        fp.write(str(np.around(qw[3], num_decimals)).rjust(float_width, ' '))
        fp.write(str(np.around(qw[4], num_decimals)).rjust(float_width, ' '))
        fp.write(str(int(dic['N1_v'][0][i])).rjust(int_width + 5, ' '))
        fp.write(str(int(dic['N2_v'][0][i])).rjust(int_width, ' '))
        fp.write(str(int(dic['N1_v'][0][i] + dic['N2_v'][0][i])).rjust(int_width, ' '))
        fp.write(str(round(dic['diff_angle'][0][i], num_decimals)).rjust(float_width, ' '))
        fp.write(str(round((norm_veca[m] + norm_vecc[n]) / 2, num_decimals)).rjust(float_width, ' '))
        fp.write(str(round((norm_vecb[m] + norm_vecd[n]) / 2, num_decimals)).rjust(float_width, ' '))
        fp.write("\n")
    fp.close()

def sort_and_save_data(l1, l2, dic, norm_veca, norm_vecb, norm_vecc, norm_vecd):
    it = list(_flatten(sorted(enumerate(dic['N_v'][0]), key=lambda x: x[1])))  
    it = it[0::2] 
    filename = "sorted_by_atomic_number.txt"
    save_data(l1, l2, dic, norm_veca, norm_vecb, norm_vecc, norm_vecd, it, filename)
    diff_angle = array(dic['diff_angle'][0])
    array_it = array(it)
    diff_angle = diff_angle[array_it]
    temp_angle = sort(list(set(dic['diff_angle'][0]))) 
    iit = []
	
    for i in range(len(temp_angle)):
        logical = (diff_angle == temp_angle[i]) 
        array_it = list(array_it[logical]) 
        iit.append(array_it)
        array_it = array(it) 
    it = _flatten(iit) 
    filename = "sorted_by_custom_angle.txt"
    save_data(l1, l2, dic, norm_veca, norm_vecb, norm_vecc, norm_vecd, it, filename) 

file1 = "../examples/Heterostructure/04_Ni111_POSCAR"
file2 = "../examples/Heterostructure/04_MoS2_POSCAR"
max_angle_mismatch = 2
max_lattice_mismatch = 0.05
angle_of_hetero = 120

supercell = (2, 2, 2)
vacuum = 15
distance = 3

translation_vectors = (-0.5, 0.5, 0)

In [32]:
atoms = read("../examples/CONVCELL.vasp")
prim_atoms = find_primitive_cell(atoms)
# view(atoms)
# write("POSCAR", atoms, direct=True, sort=True, vasp5=True)